<a href="https://www.kaggle.com/code/ajiboyetoluwalase/gen-ai-capstone-pickmylaptop-agent?scriptVersionId=236310902" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Gen AI Intensive course Capstone project - PickMyLaptop bot 


## Use Case: AI-Powered Laptop Shopping Assistant for the Everyday User

Have you ever felt overwhelmed trying to choose a new laptop — especially when you know absolutely *nothing* about processors, RAM, GPUs, or screen resolutions?  
For many people, this decision feels like a huge, frustrating milestone.

That’s where this assistant comes in.

This project introduces a smart, conversational **Laptop Sales Assistant**, powered by Generative AI and grounded in real-world data. It's designed to guide non-technical users through the laptop-buying journey — helping them make the right choice based on their **budget, preferences, and use case**.

---

### Innovation and Impact

- **Conversational Intelligence**: Users can talk to the assistant in plain English, just like chatting with a tech-savvy friend.
- **Tool-Enhanced Gen AI**: Combines Gemini's advanced reasoning with:
  - **SQL tool access** to pull accurate laptop specs and prices.
- **Personalized Recommendations**: The assistant asks clarifying questions to tailor results — whether you're a student, a designer, a remote worker, or a casual user.
- **Data-Driven Answers Only**: It never makes things up — all suggestions are based strictly on what's found in the SQL database.
- **User-Centered Design**: Built to help those who aren't tech-savvy feel confident, informed, and supported in their purchase.

---

### Real-World Use Potential

This assistant can be deployed as:
- An in-store virtual sales rep.
- A chatbot on an e-commerce site.
- A personal tech advisor for family members who always ask *"Which one should I buy?"*

By blending **accessibility**, **accuracy**, and **AI-powered guidance**, this project demonstrates how Gen AI can make technology more approachable — and shopping more human.

### Gen AI Features Used

- **Agents**: The assistant is built using Gemini Agents, which orchestrate reasoning and tool use in a structured, modular way — handling both SQL and vector search with custom tools.
- **Few-Shot Prompting**: Carefully crafted example interactions are provided to guide the model’s tone and logic, helping it give consistent and helpful responses.
- **Function Calling**: The assistant dynamically decides when to call specific tools (like querying the SQL database or retrieving from FAISS) based on the user's input — making it smart and responsive to real-time needs.



## Note: Interactive Chatbot

This notebook uses a **text-based chatbot interface** built with Python's [`input()`](https://docs.python.org/3/library/functions.html#input) function for real-time user interaction.

- The `run_bot()` function starts a loop where you can type messages and get responses from the AI assistant.
- Keep an eye out for the steps where you need to uncomment the `run_bot()` — it will not work with `Run all`, as it waits for live user input.
- If you're planning to save or export the notebook using **"Save and Run All"**, be sure to **re-comment out** the `run_bot()` call, or it will pause waiting for input.

This design provides a hands-on, interactive experience — just open the cell and try chatting with your own laptop sales assistant!


## Setup
###  Installing the needed libraries 

In [1]:
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.8 MB/s eta 0:00:00


### Importing Necessary libraries 

In [2]:
from google import genai
from google.genai import types

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split # Split data into train data and test data
from sklearn.metrics import r2_score, mean_squared_error # Metrics for Regression Analysis
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

genai.__version__

'1.7.0'

### Add your API Key

> **Note**:  
> To use the Gemini API, you’ll need to provide your **Google Generative AI API key**.  
> If you don’t have one yet, you can get it from [ai.google.dev](https://ai.google.dev/gemini-api/docs/api-key).  
> Keep your key secret and avoid sharing it in public notebooks.

In [3]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

## Importing and Cleaning the Dataset

Credits to [Khairullah Ilyas](https://www.kaggle.com/khairullahilyas) for the initial analysis in his excellent [Laptop Price Prediction Analysis notebook](https://www.kaggle.com/code/khairullahilyas/laptop-price-prediction-analysis#2.-Data-Preprocessing).

> **Note**:  
> This dataset was selected because it provides a clean, structured summary of laptop specifications and prices — ideal for powering an AI recommendation assistant.  

However, it comes with some limitations:
- **Limited size**: The dataset has a relatively small number of entries, so it may not represent all brands or price segments.
- **Missing values**: A few columns contain `NaN` entries, which can affect recommendation accuracy or query results.
- **Outliers**: There are occasional extreme values in price/specs that may skew suggestions if not handled properly.

Despite these challenges, it serves as a solid starting point to demonstrate how Gen AI can interact with structured product data in a meaningful way.
 

In [4]:
df = pd.read_csv('/kaggle/input/laptop-price-dataset-april-2024/cleaned.csv')
#Checking the dataset
df.head(1)

,Brand,Product_Description,Screen_Size,RAM,Processor,GPU,GPU_Type,Resolution,Condition,Price
0,Lenovo,Lenovo ThinkPad 14” HD Laptop PC Computer Core...,14.0,16,Intel Core i5 7th Gen.,Intel HD Graphics 520,Integrated/On-Board Graphics,NaN,Very Good - Refurbished,189.99


### Data Cleaning & Feature Engineering

To prepare the dataset for AI-based querying and recommendations, several preprocessing steps were applied to ensure consistency and usability of the data.

### Handling Missing Values
We used a **SimpleImputer** to fill missing values in key columns (`Resolution`, `GPU_Type`, `GPU`) using the **most frequent** value strategy. This ensures consistency across entries and reduces issues during querying.


In [5]:
imputer = SimpleImputer(strategy='most_frequent').set_output(transform="pandas")
df[["Resolution","GPU_Type", "GPU"]] = imputer.fit_transform(df[["Resolution","GPU_Type","GPU"]])

### Extracting Numerical Values from RAM Column

We created a function extract_numbers() to extract the first number (RAM size) from text strings like `16GB` or `8 GB DDR4`. This helps standardize RAM values for filtering or recommendations

In [6]:
# This function  extracts the first number it finds in a given text string using  the re package 
def extract_numbers(text):
    nums = re.findall(r'\d+(?:\.\d+)?', text)  # Regular expression to find numbers
    if nums:
        return int(nums[0])  # Return the first number as a integer
    else:
        return None  # Return None if no number is found

In [7]:
# Apply the function to each element in the DataFrame
df["RAM"] = df[["RAM"]].map(extract_numbers)

###  Simplifying GPU Types

GPU types were grouped into two categories:

    - `Dedicated Graphics`

    - `Integrated/On-Board Graphics` (default for anything else)

In [8]:
# Group GPU type into two groups
df["GPU_Type"] = df["GPU_Type"].apply(lambda x: "Integrated/On-Board Graphics" if x != "Dedicated Graphics" else x)

### Extracting Storage Details from Descriptions

We parsed storage information from the Product_Description column using regex to identify SSD and HDD capacities.

In [9]:
def extract_storage(text):
    """
    Extract information about storage capacity from product description.
    """
    SSD_pattern = r"(\d+\.?\d*)\s*(TB|GB)\s*(SSD)"
    HDD_pattern = r"(\d+\.?\d*)\s*(TB|GB)\s*(HDD)"
    return re.findall(SSD_pattern, text), re.findall(HDD_pattern, text)

We then normalized units (converting TB to GB) and extracted numeric values:

In [10]:
# Making of storage column from product description column
storage_df = df["Product_Description"].apply(extract_storage)
df["SSD"] = storage_df.apply(lambda x: " ".join(x[0][0]) if x[0] != [] else '0')
df["HDD"] = storage_df.apply(lambda x: " ".join(x[1][0]) if x[1] != [] else '0')
df["SSD"] = df["SSD"].str.replace("TB","000 GB").str.findall("\d").apply(lambda x: "".join(x)).astype("int64")
df["HDD"] = df["HDD"].str.replace("TB","000 GB").str.findall("\d").apply(lambda x: "".join(x)).astype("int64")
df["SSD"].unique(), df["HDD"].unique()

(array([  256,  2000,     0,   512,  1000,   238,   128,   240,    16,
           64,   120,   255,   500,   250,   480,   800,   180,  4000,
        15000,     4]),
 array([   0, 1000,  256,  500,  512, 2000]))

### Outlier Handling

We noticed a suspicious value `(15000 GB)` in the SSD column. Upon inspection, the correct value based on the product description was `256 GB`, so we replaced it accordingly.

In [11]:
# Notice above that there is a hdd of 15TB thats a suspicious value , let's check it
df.loc[(df["SSD"] == 15000)]

,Brand,Product_Description,Screen_Size,RAM,Processor,GPU,GPU_Type,Resolution,Condition,Price,SSD,HDD
1594,Asus,"ASUS Vivobook 15.6"" Touchscreen Intel 11th Gen...",15.6,8.0,Intel Core i3 11th Gen.,Intel HD Graphics 5000,Integrated/On-Board Graphics,1920 x 1080,New,375.0,15000,0


In [12]:
# Based on product description, we know that SSD size should be 256 GB
df["SSD"] = df["SSD"].replace(15000,256)

We have corrected the data format for several columns. Furthermore, some formatted columns will have empty data. Therefore, we will perform imputation with the appropriate method

In [13]:
df.isnull().sum()

Brand                  0
Product_Description    0
Screen_Size            0
RAM                    5
Processor              0
GPU                    0
GPU_Type               0
Resolution             0
Condition              0
Price                  0
SSD                    0
HDD                    0
dtype: int64

In [14]:
num_cols = df.select_dtypes(include=["float","int"]).columns
imputer = KNNImputer(n_neighbors=2).set_output(transform="pandas")
df[num_cols] = imputer.fit_transform(df[num_cols])

### These steps help create a cleaner and more reliable dataset for querying, but note:

- The dataset is relatively small, which may limit the accuracy of recommendations.

- Still, this provides a solid foundation to demonstrate Gen AI capabilities in a real-world scenario.

## Setting up the SQL Database

> This step converts the cleaned DataFrame into a structured SQL database, allowing the AI agent to query laptop specs and prices using SQL tools.


In [15]:
#Loading the sql extension into the notebook and creating the laptop_sales database
%load_ext sql
%sql sqlite:///laptop_sales.db

In [16]:
import sqlite3

# Connecting to the created laptop_sales database
conn = sqlite3.connect("laptop_sales.db")
cursor = conn.cursor()

#Creating a table for the columns in the laptop_price dataset
cursor.execute("""
CREATE TABLE IF NOT EXISTS Laptop (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Brand TEXT,
    Product_Description TEXT,
    Screen_Size TEXT,
    RAM FLOAT, 
    Processor TEXT,
    GPU TEXT,
    GPU_Type TEXT,
    Resolution TEXT,
    Condition TEXT,
    Price FLOAT,
    SSD INTEGER,
    HDD INTEGER
)
""")

# Insert the laptop_sales data into the sql database
for _, row in df.iterrows():
    cursor.execute("INSERT INTO Laptop (Brand, Product_Description, Screen_Size, RAM, Processor, GPU, GPU_Type, Resolution, Condition, Price, SSD, HDD) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", 
                   (row['Brand'], row['Product_Description'], row['Screen_Size'], row['RAM'], row['Processor'], row['GPU'], row['GPU_Type'], row['Resolution'], row['Condition'], row['Price'], row['SSD'], row['HDD']))

conn.commit()

## Setting up the Function

> This function is used to execute SQL queries and return the results, allowing the assistant to fetch relevant laptop data from the database.


In [17]:
#This function is the only function for now the agent will be able to extract information fron the database with this function
def execute_query(sql: str) -> list[list[str]]:
    """Execute an SQL statement, returning the results."""
    print(f' - DB CALL: execute_query({sql})')

    cursor = conn.cursor()

    cursor.execute(sql)
    return cursor.fetchall()


#execute_query("select * from Laptop")
execute_query("SELECT * FROM Laptop WHERE id = 3")

 - DB CALL: execute_query(SELECT * FROM Laptop WHERE id = 3)


[(3,
  'HP',
  'NEW HP 15 Laptop. 15.6" 1080p, i5-1135G7, 8GB RAM, 256GB SSD,Win 11 -15-dy2795wm',
  '15.6',
  8.0,
  'Intel Core i5 11th Gen.',
  'Intel Iris Xe Graphics',
  'Integrated/On-Board Graphics',
  '1920 x 1080',
  'New',
  369.95,
  256,
  0)]

### Defining the BOT Prompt

> This prompt sets the behavior and personality of the AI assistant. It instructs the bot to act like a friendly, knowledgeable laptop expert who uses only the SQL database for recommendations.  
> It includes **few-shot prompting** by providing clear examples of how the bot should respond, ask clarifying questions, and avoid guessing — helping guide its responses toward helpful, realistic outputs.


In [18]:
BOT_PROMPT = """You are a helpful, knowledgeable, and friendly Laptop Sales Assistant. 
Your goal is to help users find the best laptops based on their needs, preferences, and budget.

You have access to an SQL database containing laptop specifications, prices, and descriptions. 
Use this information to give accurate, helpful recommendations.
Always ask clarifying questions if the user's request is vague or missing important details (e.g., budget, intended use, preferred brands, etc.).

Provide concise, clear advice that balances performance and value for money. 
If multiple laptops match the request, suggest the top 2–3 options with brief explanations.

Do not make up data—rely only on the information provided via the SQL database or tools. 
If no match is found, kindly inform the user and suggest alternatives.

Respond in a warm and helpful tone, like a tech-savvy friend who wants the best for the user
Use list_tables_columns to see what tables are present and execute_query to issue an SQL SELECT query 
The sql table is called "Laptop"
it contains 
    Brand - "this is the brand of the laptop",
    Product_Description - "Basic description of the choosen laptop",
    Screen_Size - "the size of the laptop in inches(14,16)",
    RAM - "the size of the laptop RAM ", 
    Processor - "The processor being used by the laptop",
    GPU - "The graphical processing unit being used by the laptop",
    GPU_Type - "The type of GPU being used by the laptop",
    Resolution - "The Resolution of the laptop ",
    Condition - "The condition of the laptop at the time of sale (eg New,Used)",
    Price - "The Price at which the laptop was sold ",
    SSD - "This is the storage of the laptop"
    HDD - "This is the storage of the laptop but its slower than its counterpart(SSD)"

Example 1: Budget + Use Case

User: I'm looking for a laptop under $800 for school and some light photo editing.

Assistant: Got it! For under $800, here are two great options:

    1.Dell Inspiron 15 – Intel i5, 8GB RAM, 512GB SSD. Lightweight with a solid display for everyday tasks and basic editing.
    
    2.Acer Swift 3 – Ryzen 5, 16GB RAM, 256GB SSD. Great value for performance, especially with the extra RAM.

Would you prefer a larger screen or a more portable size?

Example 2: Vague Query

User: I want a good laptop.

Assistant: Sure! To help you better, could you tell me a bit more about:

    1.Budget: What's the maximum amount you're willing to spend?
    
    2.Intended Use: What will you primarily use the laptop for (e.g., work, gaming, school, general use, Video Editing)?
    
    3.Any preferred brands or specs (brands like Dell,Hp or acer,specs like screen size or RAM)?
    
    4.Operating System: Do you have a preferred operating system (e.g., Windows, macOS, ChromeOS)?
    
    5.Features: Are there any specific features that are important to you (e.g., long battery life, touchscreen, lots of storage)?

Example 3: Specific Requirement

User: I need a laptop with at least 16GB RAM, an i7 processor, and a 15-inch screen.

Assistant: Here are two laptops that match those specs:

    1. HP Envy 15 – Intel i7, 16GB RAM, 512GB SSD. Great for multitasking and creative work.
    
    2.Lenovo IdeaPad 5 Pro – AMD Ryzen 7 (equivalent to i7), 16GB RAM, 1TB SSD. Slightly better storage, solid performance.

Both are good picks—do you have a preferred budget range?

With that info, I can recommend something that fits you perfectly.
"""

## Set Up the Agent

> This section initializes the Gemini Flash 2.0 agent with **automatic function calling** enabled.  
> It links the assistant to the `execute_query` function and applies the custom BOT prompt to guide its behavior.  
> This setup allows the assistant to interpret user queries and decide when to run SQL commands automatically.


In [19]:
from google.genai import types
from google.api_core import retry

#using Automatic function calling for the bot 
assistant_system = [
    execute_query
]
model_name = "gemini-2.0-flash"  

client = genai.Client(api_key=GOOGLE_API_KEY)

chat = client.chats.create(
    model=model_name,
    config=types.GenerateContentConfig(
        tools=assistant_system ,
        system_instruction= BOT_PROMPT
    ),
)

## Running the Assistant Bot
> This is a sample prompt sent to the Laptop Assistant to demonstrate how it responds to a typical user request.  
> The query includes intended use (**gaming**) and a **budget** of $500.


In [20]:
response = chat.send_message('i want to get a laptop probably for work  and my budget is about $500. if you find any just tell me ')
print(f"\n{response.text}")


Okay, I can help with that! To make sure I find the best laptop for you, could you tell me what kind of work you'll be doing? For example, will you be using it for general office tasks, graphic design, programming, or something else?



### Note 

> The `run_bot()` function starts an interactive chat with the Laptop Assistant.  
> Since it uses Python’s `input()` to collect user input, it will **pause execution** when run in a notebook.

### **To use it:**
- Uncomment the `run_bot()` call to try it interactively.
- When finished, **re-comment it** if you plan to use `Run all` to avoid interruption.

```python
# run_bot()  ← Uncomment to run interactively, re-comment before running all cells


In [21]:
from IPython.display import display, Markdown

def run_bot():
    print("👋 Welcome to Laptop asistant Bot! Type 'exit' to quit.\n")

    while True:
        user_input = input("🧑 You: ")
        response = chat.send_message(user_input.lower())
        display(Markdown(response.text))
        
        if user_input.lower() == 'exit':
            print("🤖 Bot: Goodbye! 👋")
            break
#run_bot()

## More to be done – Feature Improvements

- FAISS-Based Semantic Search:
    Enable natural language queries by embedding laptop descriptions and user queries with Gemini. This allows users to search using phrases like "lightweight laptop for photo editing" and retrieve semantically similar options.

- Automated Web Scraper for Price Updates:
    Build a web scraping tool to periodically check and update laptop prices from e-commerce sites. Keeps the SQL database current and reliable.

- Laptop Comparison Tool:
    Let users compare 2–3 laptops side by side based on key specs, pricing, and descriptions. Useful for decision-making and highlighting value differences.